In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep
from bs4 import BeautifulSoup
import lxml.html as lh
from datetime import datetime
from lxml import etree

options = webdriver.ChromeOptions() 
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'})
print(driver.execute_script("return navigator.userAgent;"))
driver.get('https://www.httpbin.org/headers')

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36


In [2]:
url = "https://www.uefa.com/uefachampionsleague/match/2037765--man-city-vs-inter/"
driver.get(url)
driver.implicitly_wait(10)

In [35]:
html_content = driver.page_source

# Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')
team_names = str(soup.find('title'))
home_team = team_names.split('|')[0].split('-')[0].split('>')[1]
away_team = team_names.split('|')[0].split('-')[1]

In [72]:
# Find all <div> elements with the class "match-date"
div_match_stats = soup.find_all('div', class_='stats-module__single-stat stats-module__single-stat--comparison')

team_data = {}

team_data['Home Team'] = []
team_data['Home Team'].append(home_team)
team_data['Away Team'] = []
team_data['Away Team'].append(away_team)

import json
div_date = soup.find('div', class_="d3-react match-info-column pk-theme--light")
# print(div_date)
match = re.search(r"data-options='(.*?)'", str(div_date))

if match:
    data_options = match.group(1)
    options = json.loads(data_options)
    
    # Extract the date from the kickOffTime
    match_date = options['matchOptions']['match']['kickOffTime']['date']
    
team_data['Match Date'] = []
match_date = match_date.split('T')[0]
team_data['Match Date'].append(match_date)

for stat in div_match_stats:
    stat_item = stat.find('pk-list-stat-item')
    data_stat_type = stat_item['data-stat-type']
    home_team_stat = 'Home Team ' + data_stat_type
    away_team_stat = 'Away Team ' + data_stat_type
    home_team_data = stat_item['data']
    away_team_data = stat_item['second-data']
    
    if home_team_stat not in team_data:
        team_data[home_team_stat] = []
    if away_team_stat not in team_data:
        team_data[away_team_stat] = []
        
    team_data[home_team_stat].append(home_team_data)
    team_data[away_team_stat].append(away_team_data)

match_stats = pd.DataFrame(team_data)
match_stats.head()

,Home Team,Away Team,Match Date,Home Team ball_possession,Away Team ball_possession,Home Team passes_accuracy,Away Team passes_accuracy,Home Team passes_attempted,Away Team passes_attempted,Home Team passes_completed,...,Home Team blocked,Away Team blocked,Home Team clearance_completed,Away Team clearance_completed,Home Team yellow_cards,Away Team yellow_cards,Home Team red_cards,Away Team red_cards,Home Team fouls_committed,Away Team fouls_committed
0,Man City,Inter,2023-06-10,57,43,87,83,513,401,448,...,2,0,12,2,2,4,0,0,11,17


In [31]:
# import pandas as pd

# ['Home Team', 'Away Team', 'Home Team Possession', 'Away Team Possession', 'Home Team Pass Accuracy', 'Away Team Pass Accuracy', 'Home Team Passes Attempted', 'Away Team Passes Attempted', 'Home Team Passes Completed', 'Away Team Passes Completed', 'Home Team Distance Covered', 'Away Team Distance Covered', 'Home Team Goals', 'Away Team Goals', 'Home Team Attempts', 'Away Team Attempts', 'Home Team Attempts On Target', 'Away Team Attempts On Target', 'Home Team Attempts Off Target', 'Away Team Attempts Off Target', 'Home Team Attempts Blocked', 'Away Team Attempts Blocked', 'Home Team Attempts On Woodwork', 'Away Team Attempts On Woodwork', "Home Team Corners", "Away Team Corners", "Home Team Offsides", "Away Team Offsides", "Home Team Balls Recovered", "Away Team Balls Recovered", "Home Team Tackles", "Away Team Tackles", "Home Team Shots Blocked", "Away Team Shots Blocked", "Home Team Clearances", "Away Team Clearances","Home Team Yellow Cards", "Away Team Yellow Cards", "Home Team Red Cards", "Away Team Red Cards", "Home Team Fouls", "Away Team Fouls"])

,Home Team,Away Team,Home Team Possession,Away Team Possession,Home Team Pass Accuracy,Away Team Pass Accuracy,Home Team Passes Attempted,Away Team Passes Attempted,Home Team Passes Completed,Away Team Passes Completed,...,Home Team Shots Blocked,Away Team Shots Blocked,Home Team Clearances,Away Team Clearances,Home Team Yellow Cards,Away Team Yellow Cards,Home Team Red Cards,Away Team Red Cards,Home Team Fouls,Away Team Fouls
